In [1]:
#################### CLTV Forecast with BG-NBD and Gamma-Gamma #########################

In [2]:
#FLO wants to set a roadmap for its sales and marketing activities.
#In order for the company to make a medium-long-term plan, 
#it is necessary to estimate the potential value that the existing customers will provide to the future company.

In [3]:
#It consists of the information obtained from the past shopping behaviors of customers
#who made their last shopping from Flo as OmniChannel (both online and offline shopping) in the years 2020-2021.

In [4]:
##The Story of Data Set

# master_id : Unique customer id,
# order_channel : Which channel of the shopping platform is used (Android, ios, Desktop, Mobile),
# last_order_channel : The channel where the most recent purchase was made,
# first_order_date : Date of the customer's first purchase,
# last_order_date :Date of the customer's last purchase,
# last_order_date_online : The date of the last purchase made by the customer on the online platform,
# last_order_date_offline : The date of the last purchase made by the customer on the offline platform,
# order_num_total_ever_online :The total number of purchases made by the customer on the online platform,
# order_num_total_ever_offline :The total number of purchases made by the customer on the offline platform
# customer_value_total_ever_offline : Total fee paid by the customer for offline purchases,
# customer_value_total_ever_online : Total fee paid by the customer for online purchases,
# interested_in_categories_12 : List of categories the customer has shopped in the last 12 months

In [5]:
#Task 1: Data Preparing

In [2]:
!pip install lifetimes

  Using cached Lifetimes-0.11.3-py3-none-any.whl (584 kB)
  Using cached autograd-1.5-py3-none-any.whl (48 kB)


In [1]:
import datetime as dt
import matplotlib as plt
import pandas as pd
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

In [8]:
#Step 1: Read the flo_data_20K.csv data. Make a copy of the dataframe.

In [2]:
df = pd.read_csv(r'C:\Users\esran\Desktop\DATA SET\flo_data_20k.csv')
df.head()
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
dtype: object

In [ ]:
# Step 2: Define the required outlier_thresholds and replace_with_thresholds functions to suppress outliers.

In [5]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


In [3]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)

In [14]:
#Step 3: If the variables "order_num_total_ever_online",
#"order_num_total_ever_offline", "customer_value_total_ever_offline", 
#"customer_value_total_ever_online" have outliers, we suppress them.

In [6]:
columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline","customer_value_total_ever_online"]
for col in columns:
    replace_with_thresholds(df, col)

In [16]:
# Step 3: Omnichannel means that customers shop from both online and offline platforms.
# Create new variables for the total number of purchases and spending of each customer.

In [7]:
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [18]:
# Step 4: Examine the variable types. Change the type of variables that express date to date.

In [8]:
date_columns = [col for col in df.columns if "date" in col]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [20]:
#Task 2: Creating the CLTV Data Structure

In [21]:
#Step1: Remain 2 days after the date of the last purchase in the data set as the analysis date.

In [9]:
df["last_order_date"].max()
today_date = dt.datetime(2021, 6, 1)
type(today_date)

datetime.datetime

In [23]:
#Step2: Create a new cltvdataframe containing the values of customer_id, 
#recency_cltv_weekly, T_we ekly, frequency and monetary_cltv_avg.
#Monetary value will be expressed as average value per purchase, recency and tenure values will be expressed in weekly terms.

In [10]:
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"]- df["first_order_date"]).astype('timedelta64[D]')) / 7
cltv_df["T_weekly"] = ((today_date - df["first_order_date"]).astype('timedelta64[D]'))/7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

In [25]:
#Task 3: Establishment of BG/NBD, Gamma-Gamma Models and Calculation of CLTV

In [26]:
#Step 1:Fit the BG/NBD model

In [11]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

C:\Users\esran\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [ ]:
#estimated the expected purchases from the customers within 3 months 
#and add the exp_sales_3_month to the cltv dataframe.

In [12]:
cltv_df["exp_sales_3_month"] = bgf.predict(4*3,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

In [ ]:
#Step2: Fit the Gamma-Gamma model. 
#Estimate the average value that customers will leave and add it to the cltvdataframe as exp_average_value.

In [13]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],  # getireceği parayı tahmiin ediyoruz
                                                                cltv_df['monetary_cltv_avg'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,193.6327
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,96.6650
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,120.9676
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,67.3201
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,114.3251


In [14]:
#Step3: Calculate 6-month CLTV and add it to the dataframe with the cltv name.
#• Observe the 20 people with the highest CLTV value

In [17]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)
cltv_df["cltv"] = cltv
cltv_df.sort_values("cltv",ascending=False)[:20]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,1401.8000,1.0944,1449.0605,3327.7770
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,758.0855,1.9701,767.3606,3172.3944
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7222,1127.6115,1708.9821
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.5714,16.0000,501.8737,1.5653,506.1667,1662.6135
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,166.2246,4.6561,166.7123,1628.8874
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.4286,8.0000,601.2262,1.2655,611.4926,1623.8127
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.4286,2.0000,862.6900,0.7939,923.6800,1538.8559
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.0000,17.0000,259.8653,2.7807,262.0729,1529.2280
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,228.5300,3.0838,229.6069,1485.8192
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.8571,15.5714,3.0000,743.5867,0.8716,778.0504,1422.9997


In [16]:
#Task 4: Creating Segments by CLTV Value

In [18]:
#Step1: Divide all your customers into 4 groups (segments) according to 6-month CLTV and add the groupnames to the dataset.

In [19]:
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,193.6327,395.7332,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,96.6650,199.4307,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,120.9676,170.2242,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,67.3201,98.9455,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,114.3251,95.0117,D
